In [1]:
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import joblib
import os

# Read the data
data = pd.read_csv('project_data/train_customer_experience.csv')

# Define experience factors to analyze
experience_factors = [
    'support_tickets',
    'service_calls',
    'product_issues',
    'satisfaction_score',
    'response_time_hrs'
]

# Analyze experience factors for churned vs retained customers
churned = data[data['is_churn'] == 1]
retained = data[data['is_churn'] == 0]

# Calculate metrics for each factor
analysis = {}
for factor in experience_factors:
    analysis[factor] = {
        'churned_avg': churned[factor].mean(),
        'retained_avg': retained[factor].mean(),
        'impact_score': (churned[factor].mean() - retained[factor].mean()) / retained[factor].mean() * 100
    }

# Calculate thresholds for identifying problems
thresholds = {
    'high_tickets': data['support_tickets'].quantile(0.75),
    'low_satisfaction': data['satisfaction_score'].quantile(0.25),
    'slow_response': data['response_time_hrs'].quantile(0.75),
    'price_gap': (data['monthly_charges'] - data['competitor_price']).quantile(0.75)
}

# Analyze patterns in churned customers
issues = {
    'support_problems': len(churned[churned['support_tickets'] > thresholds['high_tickets']]),
    'satisfaction_issues': len(churned[churned['satisfaction_score'] < thresholds['low_satisfaction']]),
    'response_time_problems': len(churned[churned['response_time_hrs'] > thresholds['slow_response']]),
    'price_sensitive': len(churned[churned['monthly_charges'] - churned['competitor_price'] > thresholds['price_gap']])
}

# Generate recommendations
recommendations = []

# Analyze support and service
if analysis['support_tickets']['impact_score'] > 50:
    recommendations.append({
        'area': 'Customer Support',
        'issue': 'High number of support tickets among churned customers',
        'recommendation': 'Implement proactive support measures and improve first-contact resolution',
        'priority': 'High' if issues['support_problems'] > len(data) * 0.2 else 'Medium'
    })

# Analyze satisfaction scores
if analysis['satisfaction_score']['impact_score'] < -20:
    recommendations.append({
        'area': 'Customer Satisfaction',
        'issue': 'Low satisfaction scores strongly correlate with churn',
        'recommendation': 'Implement regular satisfaction surveys and rapid response to low scores',
        'priority': 'High' if issues['satisfaction_issues'] > len(data) * 0.2 else 'Medium'
    })

# Analyze response times
if analysis['response_time_hrs']['impact_score'] > 30:
    recommendations.append({
        'area': 'Response Time',
        'issue': 'Slow response times in customer interactions',
        'recommendation': 'Set up automated initial responses and improve support staff allocation',
        'priority': 'High' if issues['response_time_problems'] > len(data) * 0.2 else 'Medium'
    })

# Print analysis results
print("\nCustomer Experience Analysis")
print("=" * 50)

print("\nExperience Factors Analysis:")
for factor, metrics in analysis.items():
    print(f"\n{factor.replace('_', ' ').title()}:")
    print(f"  Churned customers average: {metrics['churned_avg']:.2f}")
    print(f"  Retained customers average: {metrics['retained_avg']:.2f}")
    print(f"  Impact on churn: {metrics['impact_score']:.1f}%")

print("\nKey Improvement Areas:")
for issue, count in issues.items():
    print(f"{issue.replace('_', ' ').title()}: {count} customers")

print("\nRecommendations:")
for rec in recommendations:
    print(f"\nArea: {rec['area']}")
    print(f"Issue: {rec['issue']}")
    print(f"Recommendation: {rec['recommendation']}")
    print(f"Priority: {rec['priority']}")

# Save analysis results
analysis_results = {
    'factors_analysis': analysis,
    'improvement_areas': issues,
    'recommendations': recommendations
}

os.makedirs('model_output', exist_ok=True)
joblib.dump(analysis_results, 'model_output/experience_analysis.joblib')


Customer Experience Analysis

Experience Factors Analysis:

Support Tickets:
  Churned customers average: 3.20
  Retained customers average: 0.60
  Impact on churn: 433.3%

Service Calls:
  Churned customers average: 4.20
  Retained customers average: 1.40
  Impact on churn: 200.0%

Product Issues:
  Churned customers average: 2.40
  Retained customers average: 0.20
  Impact on churn: 1100.0%

Satisfaction Score:
  Churned customers average: 6.02
  Retained customers average: 8.68
  Impact on churn: -30.6%

Response Time Hrs:
  Churned customers average: 6.84
  Retained customers average: 2.44
  Impact on churn: 180.3%

Key Improvement Areas:
Support Problems: 3 customers
Satisfaction Issues: 3 customers
Response Time Problems: 3 customers
Price Sensitive: 3 customers

Recommendations:

Area: Customer Support
Issue: High number of support tickets among churned customers
Recommendation: Implement proactive support measures and improve first-contact resolution
Priority: High

Area: Cust

['model_output/experience_analysis.joblib']